In [1]:
import cv2
import os

def extract_frames(video_path, output_dir, fps=2):
    # Create a VideoCapture object
    cap = cv2.VideoCapture(video_path)

    # Get the original frames per second (fps) of the video
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(original_fps / fps)

    frame_count = 0
    saved_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if there are no frames left

        # Save the frame if it's the right interval
        if frame_count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"{os.path.basename(video_path).split('.')[0]}_frame_{saved_frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1

        frame_count += 1

    cap.release()
    print(f"Extracted {saved_frame_count} frames from {video_path} at {fps} fps.")

def main():
    # Specify the main directory containing the videos
    main_directory = r"C:\Users\red\Documents\videos_weapons"  # Current directory

    # Loop through all directories and subdirectories
    for root, dirs, files in os.walk(main_directory):
        for filename in files:
            if filename.endswith(('.mp4', '.avi', '.mov', '.mkv')):  # Add more formats if needed
                video_file = os.path.join(root, filename)
                output_directory = root  # Save frames in the same directory as the video

                extract_frames(video_file, output_directory)

                # Delete the original video file
                if os.path.exists(video_file):
                    os.remove(video_file)
                    print(f"Deleted original video: {video_file}")
                else:
                    print(f"Video file not found: {video_file}")

if __name__ == "__main__":
    main()

Extracted 43 frames from C:\Users\red\Documents\videos_weapons\Ijram fess.mp4 at 2 fps.
Deleted original video: C:\Users\red\Documents\videos_weapons\Ijram fess.mp4
Extracted 34 frames from C:\Users\red\Documents\videos_weapons\Ijram italia.mp4 at 2 fps.
Deleted original video: C:\Users\red\Documents\videos_weapons\Ijram italia.mp4
Extracted 144 frames from C:\Users\red\Documents\videos_weapons\Ijram rabat.mp4 at 2 fps.
Deleted original video: C:\Users\red\Documents\videos_weapons\Ijram rabat.mp4
Extracted 72 frames from C:\Users\red\Documents\videos_weapons\Ijram sla.mp4 at 2 fps.
Deleted original video: C:\Users\red\Documents\videos_weapons\Ijram sla.mp4
Extracted 118 frames from C:\Users\red\Documents\videos_weapons\khemisset 2021.mp4 at 2 fps.
Deleted original video: C:\Users\red\Documents\videos_weapons\khemisset 2021.mp4
Extracted 59 frames from C:\Users\red\Documents\videos_weapons\salé ijram.mp4 at 2 fps.
Deleted original video: C:\Users\red\Documents\videos_weapons\salé ijram.

In [3]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:

import cv2
from ultralytics import YOLO

# Charger le modèle YOLO
model = YOLO('sa.pt')

class_list = model.names

# Ouvrir la caméra (0 est l'index de la caméra par défaut)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Effectuer la détection et le suivi des objets
    results = model.track(frame, persist=True)

    if results[0].boxes.data is not None:
        boxes = results[0].boxes.xyxy
        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int()
        else:
            track_ids = []
        class_indices = results[0].boxes.cls.int()
        confidences = results[0].boxes.conf

        for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
            x1, y1, x2, y2 = map(int, box)

            class_name = class_list[int(class_idx)]

            # Ajouter le texte avec le nom de la classe et l'ID
            cv2.putText(frame, f"ID: {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
            # Dessiner la boîte de détection
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Afficher la frame avec les objets détectés
    cv2.imshow("Object Tracking", frame)

    # Quitter la boucle si la touche 'q' est pressée
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la caméra et fermer les fenêtres ouvertes
cap.release()
cv2.destroyAllWindows()